# API DATA DOWNLOAD MODULE

Made by: Prabin Raj Shrestha

<a href="https://colab.research.google.com/drive/1F-JP9Tk892XnlnmM_PpzEU0IuwyMsGnZ" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

## Packages

In [ ]:
!pip install full-fred

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 601.0 kB/s eta 0:00:00


In [ ]:
import requests
import json
from full_fred.fred import Fred
from google.colab import files
import pandas as pd
from tqdm import tqdm  # Import tqdm library

---


### API Key
Queries to FRED web service require an API key. FRED has [free API keys available with an account (also free)](https://research.stlouisfed.org/useraccount/apikey)

make at least 3 API Keys and add them to the list below

In [ ]:
API_Key_l = ['<api key1>', '<api key2>', '<api key3>']

In [ ]:
# @title API manager

from full_fred.fred import Fred  # Import Fred class from full_fred package

class api_manager():
  def __init__(self, api_l: list):
    # Initialize api_manager with a list of API keys
    self._update_api_l(api_l)
    # Set the filename for storing the API key
    self.api_fn = 'api_key.txt'

  def _update_api_l(self, api_l: list):
    # Update list of API keys and set counters
    self.api_l = api_l
    self.n_api = len(api_l)
    self.n = 0
    self.cycle_api()

  def n_up(self):
    # Increment counter for cycling through API keys
    self.n += 1
    # Reset counter to 0 if it reaches the end of the list
    if self.n == self.n_api:
      self.n = 0

  def get_api(self, n = None):
    # Get API key by index, or return current API key if index is None
    if n is None:
      return self.api
    else:
      return self.api_l[n]

  def update_api_f(self):
    # Update the API key file with the current API key
    api = self.get_api()
    with open(self.api_fn, 'w') as file:
      file.write(api)
    return self.api_fn

  def cycle_api(self):
    # Cycle to the next API key and return it
    self.api = self.get_api(self.n)
    self.n_up()
    return (self.api)

  def cycle_api_file(self, ):
    # Cycle to the next API key and update the API key file
    self.cycle_api()
    return self.update_api_f()

  def update_fred(self):
    # Update Fred object with the current API key from the API key file
    return Fred(self.update_api_f())

  def cycle_fred(self):
    # Cycle to the next API key and update Fred object
    return Fred(self.cycle_api_file())


In [ ]:
# @title Data Download Object

class fred_df_manager:
  def __init__(self, config_dict):
    # Initialize config_dict
    self.config_dict = {}
    # Update config_dict with provided configuration
    self.update_config(config_dict)

  def update_config(self, config_dict):
    # Check if anything import ant is missing
    check_l = ['api_url', 'date', 'series_id', 'metric_name', 'api_manager']
    missing_keys = [key for key in check_l if key not in list(self.config_dict.keys()) + list(config_dict.keys())]

    # If missing keys are found, raise ValueError
    if len(missing_keys) != 0:
      for key in missing_keys:
        print(f'Missing: {key} from config')
      raise ValueError('Missing keys in config')

    # Update config
    for key in config_dict:
      self.config_dict[key] = config_dict[key]

    # URL to get series id
    self.api_url_s = self.config_dict['api_url']
    # Date
    self.date = self.config_dict['date']
    # Series
    self.series_id = self.config_dict['series_id']
    # Metric Name
    self.metric_name = self.config_dict['metric_name']
    # api_manager
    self.api_manager = self.config_dict['api_manager']

  def api_json(self, url_s, payload):
    # Send API request and return JSON response
    r = requests.get(url_s, params = payload)
    return r.json()


  def get_info(self):
    # Payload
    payload = {'api_key': self.api_manager.get_api()
              , 'file_type': 'json'
              , 'series_id': self.series_id
              , 'date': self.date}

    # info dictionary
    info_dict = self.api_json(self.api_url_s, payload)['meta']
    for key in info_dict.keys():
      v = info_dict[key]
      if type(v) in [str, int, float]:
        print(f'{key}: {v}')
    return info_dict

  def get_data_fred(self, sid, tries: int = 3):
    # Attempt to download data with specified number of tries
    for n_try in range(tries):
      try:
        # Get series data DataFrame
        data_df = self.fred.get_series_df(sid)
        return data_df
      except:
        # Cycle to next API key if download fails
        self.fred = self.api_manager.cycle_fred()
        print(f"Switching Keys: {n_try}")
        pass
    raise ValueError('Unable to download data')

  def get_data(self, sid, tries: int = 3):
    # Update fred attribute and return data
    self.fred = self.api_manager.update_fred()
    return self.get_data_fred(sid, tries)

  def download_data(self, rename_dict= None, tries: int = 3):
    # Retrieve information dictionary
    self.info_dict = self.get_info()
    self.fred = self.api_manager.update_fred()

    # Define column list for DataFrame
    col_l = ['date', 'region', self.metric_name]

    # Define dictionary to rename columns in DataFrame
    rename_dict = {} if rename_dict is None else rename_dict

    # Initialize list to store DataFrames
    data_l = []

    # Get the total number of iterations
    total_iterations = len(self.info_dict['data'][self.date])

    # Iterate over data in information dictionary with tqdm for progress bar
    # for data_dict in self.info_dict['data'][self.date]:
    for data_dict in tqdm(self.info_dict['data'][self.date], total=total_iterations, desc='Downloading data'):
      # Download data using series_id and specified number of tries
      data_df = self.get_data_fred(data_dict['series_id'], tries)

      # Rename columns in DataFrame
      data_df = data_df.rename(columns=rename_dict)

      # Add additional information from data_dict to DataFrame
      for key in data_dict.keys():
          data_df[key] = data_dict[key]

      # Append DataFrame to list
      data_l.append(data_df)

    # Concatenate the list of DataFrames into a single DataFrame
    combined_df = pd.concat(data_l, ignore_index=True)

    # return
    return combined_df




---

# Unemployment Rate

In [ ]:
# Settings
snapshot_date = '2024-02-01'
series_id = 'WVMERC0URN'
metric_name = 'unemployment_rate'


# API Manager
am = api_manager(API_Key_l)

# CONFIG
config_dict = {'api_url': 'https://api.stlouisfed.org/geofred/series/data?'
               , 'date': snapshot_date
               , 'series_id': series_id
               , 'metric_name': metric_name
               , 'api_manager': am
              }

# download manager
dl_manager = fred_df_manager(config_dict)

# download
unemployment_rate_df = dl_manager.download_data({'value': metric_name})
unemployment_rate_df.head()

title: 2024 February Unemployment Rate by County (Percent)
region: county
seasonality: Not Seasonally Adjusted
units: Percent
frequency: Monthly


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


...

Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


,realtime_start,realtime_end,date,unemployment_rate,region,code,value,series_id
0,2024-04-21,2024-04-21,1990-01-01,1.8,"Manassas Park City, VA",51685,2.5,VAMANA5URN
1,2024-04-21,2024-04-21,1990-02-01,2.0,"Manassas Park City, VA",51685,2.5,VAMANA5URN
2,2024-04-21,2024-04-21,1990-03-01,1.7,"Manassas Park City, VA",51685,2.5,VAMANA5URN
3,2024-04-21,2024-04-21,1990-04-01,2.1,"Manassas Park City, VA",51685,2.5,VAMANA5URN
4,2024-04-21,2024-04-21,1990-05-01,2.0,"Manassas Park City, VA",51685,2.5,VAMANA5URN


In [ ]:
# Cleaning
col_l = ['date', 'year', 'month', 'country', 'region', 'state', 'county', 'code', metric_name]

unemployment_rate_df['date'] = pd.to_datetime(unemployment_rate_df['date'])
unemployment_rate_df['year'] = unemployment_rate_df['date'].dt.year
unemployment_rate_df['month'] = unemployment_rate_df['date'].dt.month
unemployment_rate_df['date'] = unemployment_rate_df['date'].dt.date

unemployment_rate_df['country'] = 'USA'
unemployment_rate_df['state'] = unemployment_rate_df['region'].apply(lambda x: str(x).split(',')[-1].strip())
unemployment_rate_df['county'] = unemployment_rate_df['region'].apply(lambda x: str(x).split(',')[0].strip())
unemployment_rate_df = unemployment_rate_df[col_l]

unemployment_rate_df.head()

,date,year,month,country,region,state,county,code,unemployment_rate
0,1990-01-01,1990,1,USA,"Manassas Park City, VA",VA,Manassas Park City,51685,1.8
1,1990-02-01,1990,2,USA,"Manassas Park City, VA",VA,Manassas Park City,51685,2.0
2,1990-03-01,1990,3,USA,"Manassas Park City, VA",VA,Manassas Park City,51685,1.7
3,1990-04-01,1990,4,USA,"Manassas Park City, VA",VA,Manassas Park City,51685,2.1
4,1990-05-01,1990,5,USA,"Manassas Park City, VA",VA,Manassas Park City,51685,2.0


In [ ]:
# download data
fn = f'{metric_name}.csv'
unemployment_rate_df.to_csv(fn, index=False)
files.download(fn)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
---
---

In [ ]:
# Settings
snapshot_date = '2024-02-01'
series_id = 'MELIPRMMCOUNTY6059'
metric_name = 'market_hotness_prec_change'


# API Manager
am = api_manager(API_Key_l)

# CONFIG
config_dict = {'api_url': 'https://api.stlouisfed.org/geofred/series/data?'
               , 'date': snapshot_date
               , 'series_id': series_id
               , 'metric_name': metric_name
               , 'api_manager': am
              }

# download manager
dl_manager = fred_df_manager(config_dict)

# # test
# market_hotness_rate_df = dl_manager.get_data(series_id)


# # download
market_hotness_rate_df = dl_manager.download_data({'value': metric_name})

# PREVIEW
market_hotness_rate_df.head()

title: 2024 February Market Hotness: Median Listing Price by County (Percent Change)
region: county
seasonality: Not Seasonally Adjusted
units: Percent Change
frequency: Monthly


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


,realtime_start,realtime_end,date,market_hotness_prec_change,region,code,value,series_id
0,2024-04-21,2024-04-21,2017-07-01,.,"Cumberland County, ME",23005,0.900901,MELIPRMMCOUNTY23005
1,2024-04-21,2024-04-21,2017-08-01,-0.6941327,"Cumberland County, ME",23005,0.900901,MELIPRMMCOUNTY23005
2,2024-04-21,2024-04-21,2017-09-01,-2.6290166,"Cumberland County, ME",23005,0.900901,MELIPRMMCOUNTY23005
3,2024-04-21,2024-04-21,2017-10-01,-0.1571429,"Cumberland County, ME",23005,0.900901,MELIPRMMCOUNTY23005
4,2024-04-21,2024-04-21,2017-11-01,-1.2734297,"Cumberland County, ME",23005,0.900901,MELIPRMMCOUNTY23005


In [ ]:
# Cleaning
col_l = ['date', 'year', 'month', 'country', 'region', 'state', 'county', 'code', metric_name]

market_hotness_rate_df['date'] = pd.to_datetime(market_hotness_rate_df['date'])
market_hotness_rate_df['year'] = market_hotness_rate_df['date'].dt.year
market_hotness_rate_df['month'] = market_hotness_rate_df['date'].dt.month
market_hotness_rate_df['date'] = market_hotness_rate_df['date'].dt.date

market_hotness_rate_df['country'] = 'USA'
market_hotness_rate_df['state'] = market_hotness_rate_df['region'].apply(lambda x: str(x).split(',')[-1].strip())
market_hotness_rate_df['county'] = market_hotness_rate_df['region'].apply(lambda x: str(x).split(',')[0].strip())
market_hotness_rate_df = market_hotness_rate_df[col_l]

market_hotness_rate_df.head()

,date,year,month,country,region,state,county,code,market_hotness_prec_change
0,2017-07-01,2017,7,USA,"Cumberland County, ME",ME,Cumberland County,23005,.
1,2017-08-01,2017,8,USA,"Cumberland County, ME",ME,Cumberland County,23005,-0.6941327
2,2017-09-01,2017,9,USA,"Cumberland County, ME",ME,Cumberland County,23005,-2.6290166
3,2017-10-01,2017,10,USA,"Cumberland County, ME",ME,Cumberland County,23005,-0.1571429
4,2017-11-01,2017,11,USA,"Cumberland County, ME",ME,Cumberland County,23005,-1.2734297


In [ ]:
# download data
fn = f'{metric_name}.csv'
market_hotness_rate_df.to_csv(fn, index=False)
files.download(fn)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
---
---

In [ ]:
# Settings
snapshot_date = '2024-02-01'
series_id = 'MELIPRCOUNTY6059'
metric_name = 'market_hotness'


# API Manager
am = api_manager(API_Key_l)

# CONFIG
config_dict = {'api_url': 'https://api.stlouisfed.org/geofred/series/data?'
               , 'date': snapshot_date
               , 'series_id': series_id
               , 'metric_name': metric_name
               , 'api_manager': am
              }

# download manager
dl_manager = fred_df_manager(config_dict)

# # test
# market_hotness_rate_df = dl_manager.get_data(series_id)


# # download
data_df = dl_manager.download_data({'value': metric_name})

# PREVIEW
data_df.head()

title: 2024 February Market Hotness: Median Listing Price by County (U.S. Dollars)
region: county
seasonality: Not Seasonally Adjusted
units: U.S. Dollars
frequency: Monthly


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


,realtime_start,realtime_end,date,market_hotness,region,code,value,series_id
0,2024-04-21,2024-04-21,2016-07-01,.,"Sebastian County, AR",05131,279950.0,MELIPRCOUNTY5131
1,2024-04-21,2024-04-21,2016-08-01,.,"Sebastian County, AR",05131,279950.0,MELIPRCOUNTY5131
2,2024-04-21,2024-04-21,2016-09-01,.,"Sebastian County, AR",05131,279950.0,MELIPRCOUNTY5131
3,2024-04-21,2024-04-21,2016-10-01,.,"Sebastian County, AR",05131,279950.0,MELIPRCOUNTY5131
4,2024-04-21,2024-04-21,2016-11-01,.,"Sebastian County, AR",05131,279950.0,MELIPRCOUNTY5131


In [ ]:
# Cleaning
col_l = ['date', 'year', 'month', 'country', 'region', 'state', 'county', 'code', metric_name]

data_df['date'] = pd.to_datetime(data_df['date'])
data_df['year'] = data_df['date'].dt.year
data_df['month'] = data_df['date'].dt.month
data_df['date'] = data_df['date'].dt.date

data_df['country'] = 'USA'
data_df['state'] = data_df['region'].apply(lambda x: str(x).split(',')[-1].strip())
data_df['county'] = data_df['region'].apply(lambda x: str(x).split(',')[0].strip())


market_hotness_df = data_df[col_l]

market_hotness_df.head()

,date,year,month,country,region,state,county,code,market_hotness
0,2016-07-01,2016,7,USA,"Sebastian County, AR",AR,Sebastian County,05131,.
1,2016-08-01,2016,8,USA,"Sebastian County, AR",AR,Sebastian County,05131,.
2,2016-09-01,2016,9,USA,"Sebastian County, AR",AR,Sebastian County,05131,.
3,2016-10-01,2016,10,USA,"Sebastian County, AR",AR,Sebastian County,05131,.
4,2016-11-01,2016,11,USA,"Sebastian County, AR",AR,Sebastian County,05131,.


In [ ]:
# download data
fn = f'{metric_name}.csv'
market_hotness_df.to_csv(fn, index=False)
files.download(fn)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
---
---

In [ ]:
# Settings
snapshot_date = r'2023-01-01'
series_id = 'CASANF0POP'
metric_name = 'population_annual'


# API Manager
am = api_manager(API_Key_l)

# CONFIG
config_dict = {'api_url': 'https://api.stlouisfed.org/geofred/series/data?'
               , 'date': snapshot_date
               , 'series_id': series_id
               , 'metric_name': metric_name
               , 'api_manager': am
              }

# download manager
dl_manager = fred_df_manager(config_dict)


# # test
# data_df = dl_manager.get_data(series_id)


# # download
data_df = dl_manager.download_data({'value': metric_name})

# PREVIEW
data_df.head()

title: 2023 Resident Population by County (Thousands of Persons)
region: county
seasonality: Not Seasonally Adjusted
units: Thousands of Persons
frequency: Annual


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


...

Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


,realtime_start,realtime_end,date,population_annual,region,code,value,series_id
0,2024-04-21,2024-04-21,1970-01-01,5.199,"Rappahannock County, VA",51157,7.414,VARAPP7POP
1,2024-04-21,2024-04-21,1971-01-01,5.200,"Rappahannock County, VA",51157,7.414,VARAPP7POP
2,2024-04-21,2024-04-21,1972-01-01,5.300,"Rappahannock County, VA",51157,7.414,VARAPP7POP
3,2024-04-21,2024-04-21,1973-01-01,5.200,"Rappahannock County, VA",51157,7.414,VARAPP7POP
4,2024-04-21,2024-04-21,1974-01-01,5.400,"Rappahannock County, VA",51157,7.414,VARAPP7POP


In [ ]:
# Cleaning
col_l = ['date', 'year', 'country', 'region', 'state', 'county', 'code', metric_name]

data_df['date'] = pd.to_datetime(data_df['date'])
data_df['year'] = data_df['date'].dt.year
# data_df['month'] = data_df['date'].dt.month
data_df['date'] = data_df['date'].dt.date

data_df['country'] = 'USA'
data_df['state'] = data_df['region'].apply(lambda x: str(x).split(',')[-1].strip())
data_df['county'] = data_df['region'].apply(lambda x: str(x).split(',')[0].strip())


population_df = data_df[col_l].drop_duplicates()

population_df.head()

,date,year,country,region,state,county,code,population_annual
0,1970-01-01,1970,USA,"Rappahannock County, VA",VA,Rappahannock County,51157,5.199
1,1971-01-01,1971,USA,"Rappahannock County, VA",VA,Rappahannock County,51157,5.200
2,1972-01-01,1972,USA,"Rappahannock County, VA",VA,Rappahannock County,51157,5.300
3,1973-01-01,1973,USA,"Rappahannock County, VA",VA,Rappahannock County,51157,5.200
4,1974-01-01,1974,USA,"Rappahannock County, VA",VA,Rappahannock County,51157,5.400


In [ ]:
# download data
fn = f'{metric_name}.csv'
population_df.to_csv(fn, index=False)
files.download(fn)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
---
---

In [ ]:
# Settings
snapshot_date = r'2023-10-01'
series_id = 'EQFXSUBPRIME036061'
metric_name = 'Equifax_Subprime_Credit_Population'


# API Manager
am = api_manager(API_Key_l)

# CONFIG
config_dict = {'api_url': 'https://api.stlouisfed.org/geofred/series/data?'
               , 'date': snapshot_date
               , 'series_id': series_id
               , 'metric_name': metric_name
               , 'api_manager': am
              }

# download manager
dl_manager = fred_df_manager(config_dict)


# # test
# data_df = dl_manager.get_data(series_id)


# # download
data_df = dl_manager.download_data({'value': metric_name})

# PREVIEW
data_df.head()

title: 2023 Q4 Equifax Subprime Credit Population by County (Percent)
region: county
seasonality: Not Seasonally Adjusted
units: Percent
frequency: Quarterly


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


...

Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


,realtime_start,realtime_end,date,Equifax_Subprime_Credit_Population,region,code,value,series_id
0,2024-04-21,2024-04-21,1999-01-01,.,"Martin County, TX",48317,32.105263,EQFXSUBPRIME048317
1,2024-04-21,2024-04-21,1999-04-01,.,"Martin County, TX",48317,32.105263,EQFXSUBPRIME048317
2,2024-04-21,2024-04-21,1999-07-01,.,"Martin County, TX",48317,32.105263,EQFXSUBPRIME048317
3,2024-04-21,2024-04-21,1999-10-01,.,"Martin County, TX",48317,32.105263,EQFXSUBPRIME048317
4,2024-04-21,2024-04-21,2000-01-01,.,"Martin County, TX",48317,32.105263,EQFXSUBPRIME048317


In [ ]:
# Cleaning
col_l = ['date', 'year', 'quarter', 'country', 'region', 'state', 'county', 'code', metric_name]

data_df['date'] = pd.to_datetime(data_df['date'])
data_df['year'] = data_df['date'].dt.year
data_df['quarter'] = data_df['date'].dt.quarter
# data_df['month'] = data_df['date'].dt.month
data_df['date'] = data_df['date'].dt.date

data_df['country'] = 'USA'
data_df['state'] = data_df['region'].apply(lambda x: str(x).split(',')[-1].strip())
data_df['county'] = data_df['region'].apply(lambda x: str(x).split(',')[0].strip())


Equifax_Subprime_Credit_df = data_df[col_l].drop_duplicates()

Equifax_Subprime_Credit_df.head()

,date,year,quarter,country,region,state,county,code,Equifax_Subprime_Credit_Population
0,1999-01-01,1999,1,USA,"Martin County, TX",TX,Martin County,48317,.
1,1999-04-01,1999,2,USA,"Martin County, TX",TX,Martin County,48317,.
2,1999-07-01,1999,3,USA,"Martin County, TX",TX,Martin County,48317,.
3,1999-10-01,1999,4,USA,"Martin County, TX",TX,Martin County,48317,.
4,2000-01-01,2000,1,USA,"Martin County, TX",TX,Martin County,48317,.


In [ ]:
# download data
fn = f'{metric_name}.csv'
Equifax_Subprime_Credit_df.to_csv(fn, index=False)
files.download(fn)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
---
---

In [ ]:
# Settings
snapshot_date = r'2024-02-01'
series_id = 'ORMULT1LFN'
metric_name = 'Civilian_Labor_Force'


# API Manager
am = api_manager(API_Key_l)

# CONFIG
config_dict = {'api_url': 'https://api.stlouisfed.org/geofred/series/data?'
               , 'date': snapshot_date
               , 'series_id': series_id
               , 'metric_name': metric_name
               , 'api_manager': am
              }

# download manager
dl_manager = fred_df_manager(config_dict)


# # test
# data_df = dl_manager.get_data(series_id)


# # download
data_df = dl_manager.download_data({'value': metric_name})

# PREVIEW
data_df.head()

title: 2024 February Civilian Labor Force by County (Persons)
region: county
seasonality: Not Seasonally Adjusted
units: Persons
frequency: Monthly


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


...

Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


Error Message: Too Many Requests.  Exceeded Rate Limit
Switching Keys: 0


In [ ]:
# Cleaning
col_l = ['date', 'year', 'quarter', 'country', 'region', 'state', 'county', 'code', metric_name]

data_df['date'] = pd.to_datetime(data_df['date'])
data_df['year'] = data_df['date'].dt.year
data_df['quarter'] = data_df['date'].dt.quarter
# data_df['month'] = data_df['date'].dt.month
data_df['date'] = data_df['date'].dt.date

data_df['country'] = 'USA'
data_df['state'] = data_df['region'].apply(lambda x: str(x).split(',')[-1].strip())
data_df['county'] = data_df['region'].apply(lambda x: str(x).split(',')[0].strip())


Civilian_Labor_Force_df = data_df[col_l].drop_duplicates()

Civilian_Labor_Force_df.head()

In [ ]:
# download data
fn = f'{metric_name}.csv'
Civilian_Labor_Force_df.to_csv(fn, index=False)
files.download(fn)

---
---
---